### Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset. 
### Probar términos de interés y explicar similitudes en el espacio de embeddings. 
### Intentar plantear y probar tests de analogías. 
### Graficar los embeddings resultantes.
### Sacar conclusiones.

In [4]:
import pandas as pd
from gensim.models import Word2Vec
# from keras.rc.legacy.preprocessing.text import text_to_word_sequence
# from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.legacy.preprocessing.text import text_to_word_sequence

### Datos
Utilizaremos como dataset que contiene reseñas de usuarios de amazon a productos de software. (https://snap.stanford.edu/data/web-Amazon.html)

In [5]:
software_txt = pd.read_json('data/Software.json', lines=True)
software_txt = software_txt['reviewText']

In [6]:
software_txt[1]

'I am really enjoying this book with the worksheets that make you review your goals, what to do when you do not make it, it reminds me  of my human sexuality classwork.'

### 1 - Preprocesamiento

In [8]:
sequence_token = []

for txt in software_txt:
    try:
        sequence_token.append(text_to_word_sequence(txt))
    except Exception as e:
        continue

In [9]:
print(sequence_token[1])

['i', 'am', 'really', 'enjoying', 'this', 'book', 'with', 'the', 'worksheets', 'that', 'make', 'you', 'review', 'your', 'goals', 'what', 'to', 'do', 'when', 'you', 'do', 'not', 'make', 'it', 'it', 'reminds', 'me', 'of', 'my', 'human', 'sexuality', 'classwork']


In [10]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### 2 - Crear los vectores (word2vec)

In [25]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=10,       # dimensionalidad de los vectores
                     negative=15,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)
                     # alpha=0.025,
                     # min_alpha=0.0001)

In [26]:
w2v_model.build_vocab(sequence_token)

In [27]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 459370


In [28]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 39347


### 3 - Entrenar embeddings

In [29]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sequence_token,
                 total_examples=w2v_model.corpus_count,
                 epochs=15,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 49192476.0
Loss after epoch 1: 22495276.0
Loss after epoch 2: 7025384.0
Loss after epoch 3: 6971984.0
Loss after epoch 4: 6928584.0
Loss after epoch 5: 6813208.0
Loss after epoch 6: 6801208.0
Loss after epoch 7: 6745992.0
Loss after epoch 8: 6707840.0
Loss after epoch 9: 6650904.0
Loss after epoch 10: 6571560.0
Loss after epoch 11: 1313312.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0


(404837506, 540308520)

In [36]:
w2v_model.save('model/software_model.pkl')

In [118]:
w2v_model = Word2Vec.load('model/software_model.pkl')

### 4 - Ensayar

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=['great'], topn=5)

[('wonderfull', 0.9832247495651245),
 ('fantastic', 0.9801145195960999),
 ('geat', 0.974557101726532),
 ('terrific', 0.9732329845428467),
 ('gr8', 0.9693342447280884)]

Como se puede observar, buscamos las palabras más similares a 'great', obteniendo palabras tales como 'fantastic', 'terrific', 'wonderfull', etc, lo que significda que agrupa palabras que tienen significados positivos similares en las reviews. Este resultado nos indica que el modelo aprendió patrones linguistico de los corpus.

In [31]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=['green'], topn=5)

[('yellow', 0.9887784719467163),
 ('white', 0.9818674325942993),
 ('bubble', 0.9814797043800354),
 ('grey', 0.9793028831481934),
 ('purple', 0.9727424383163452)]

De los resultados de obtener las palabras más similares a 'green' observamos que agrupas palabras que describen colores.

In [32]:
w2v_model.wv.similarity('software', 'application')

0.8697883

Observando similitudes entre palabras

In [33]:
w2v_model.wv.similarity('price', 'discount')

0.9171343

El valor de similitud 0.912 indica que el modelo ha capturado bien las relaciones semánticas en las reviews. Esto tiene mucho sentido, ya que en el contexto de reviews de productos, es común que las palabras relacionadas con precios y descuentos aparezcan juntas en frases.

##### Pruebas de analogía

In [34]:
tests = [
    ("good", "excellent", "bad"),  
    ("product", "quality", "price"),
    ("user", "password", 'license')
]

for a, b, c in tests:
    if all(word in w2v_model.wv for word in [a, b, c]):
        result = w2v_model.wv.most_similar(positive=[b, c], negative=[a])
        print(f"{a} -> {b} :: {c} -> {result[0][0]}")
    else:
        print(f"Uno o más términos ({a}, {b}, {c}) no están en el vocabulario.")

good -> excellent :: bad -> horrendous
product -> quality :: price -> ratio
user -> password :: license -> keycode


- El modelo identifica que "excellent" es un superlativo de "good", una relación positiva. Para el término "bad", el modelo sugiere "horrendous", que intensifica negativamente el significado de "bad". Esto indica que el modelo está capturando la estructura de intensificación semántica.
- Acá la relación no es tan clara. "Product" tiene calidad, pero "Price" no tiene "ratio" de la misma manera.
- En el tercer caso, el modelo ha devuelto una relación bastante acertada, ya que un usuario accede con una contraseña y una licencia se asocia con un código de activación (keycode).

#### Podemos concluir que el modelo aprendió correctamente las conexiones entre estas palabras. 

### 5 - Visualizar agrupación de vectores

In [37]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [39]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=400
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

En la visualización 2D de los embeddings, se observa que términos 'after', 'before', 'when', 'once' y 'then', están relacionadas con el orden de los eventos en el tiempo, lo cual tiene sentido que sus embeddings estén cercanos en el espacio vectorial.

In [40]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

En la visualización 3D, se observó que los verbos modales y sus formas negativas quedaron agrupados en el espacio vectorial. Términos como 'can', 'would', 'may', 'should', 'must' y sus contrapartes negativas ('can't', 'don't', 'wouldn't', 'won't', 'cannot') demostraron una organización coherente, lo que indica que el modelo pudo capturar relaciones gramaticales y de uso común en el lenguaje.

### Conclusiones
A partir de los embeddings obtenidos, se realizaron pruebas de analogía y exploración de vecinos semánticos, arrojando resultados coherentes con la intuición lingüística. Por ejemplo, relaciones como "good -> excellent :: bad -> horrendous" o "user -> password :: license -> keycode" evidencian que el modelo logró captar correctamente asociaciones de significado y estructuras propias del lenguaje natural.

La visualización de los embeddings mediante técnicas de reducción de dimensión (t-SNE, PCA) también permitió verificar agrupamientos lógicos de palabras por tema, lo cual refuerza la calidad semántica de los vectores obtenidos.